In [1]:
%matplotlib inline

import os
from pyspark import SparkContext, SparkConf
import json
# SparkContext.setSystemProperty('spark.executor.memory', '16g')
# os.environ['PYSPARK_PYTHON'] = '/nas/home/ngarg/anaconda3/bin/python'
# os.environ['PYSPARK_PYTHON_DRIVER'] = '/nas/home/ngarg/anaconda3/bin/python'

In [7]:
config = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=config)

In [8]:
sc.getConf().getAll()

[('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.cores.max', '3'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.memory', '8g'),
 ('spark.driver.port', '39017'),
 ('spark.executor.cores', '3'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'ckg03'),
 ('spark.app.id', 'local-1580721986125')]

In [2]:
appName = 'assignment1'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

In [9]:
BUSINESS_JSON_PATH = './data/business.json'
REVIEW_JSON_PATH = './data/review.json'

In [10]:
jsonRDD = sc.textFile(REVIEW_JSON_PATH)

In [11]:
jsonRDD.getNumPartitions()

160

In [5]:
# Task 1 A
jsonRDD.map(lambda line: json.loads(line)).map(lambda item: (1,1)).reduceByKey(lambda a, b: a+b).collect()

[(1, 6685900)]

In [6]:
# Task 1 B
def function(line):
    j = json.loads(line)
    if(j["date"].partition('-')[0].strip() == "2018"):
        return True
    
    return False
    
jsonRDD.filter(function).map(lambda line: (1, 1)).reduceByKey(lambda a,b: a+b).collect()

[(1, 1177662)]

In [7]:
jsonRDD.filter(function).count()

1177662

In [ ]:
def function(partition):
    temp = set()
    for line in partition:
        j = json.loads(line)
        temp.add(j['user_id'])
    return list(temp)

jsonRDD.mapPartitions(function).distinct().count()

In [13]:
def function(line):
    j = json.loads(line)
    return j.get('user_id')

jsonRDD.map(function).distinct().count()

1637138

In [12]:
jsonRDD.map(json.loads).map(lambda x: x['user_id']).distinct().count()

1637138

In [13]:
jsonRDD.map(json.loads).map(lambda x: (x['user_id'], 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda x: x[1], ascending=False).take(10)

[('CxDOIDnH8gp9KXzpBHJYXw', 4129),
 ('bLbSNkLggFnqwNNzzq-Ijw', 2354),
 ('PKEzKWv_FktMm2mGPjwd0Q', 1822),
 ('ELcQDlf69kb-ihJfxZyL0A', 1764),
 ('DK57YibC5ShBmqQl97CKog', 1727),
 ('U4INQZOPSUaj8hMjLlZ3KA', 1559),
 ('QJI9OSEn6ujRCtrX06vs1w', 1496),
 ('d_TBs6J3twMy9GChqUEXkg', 1360),
 ('hWDybu_KvYLSdEFzGrniTw', 1355),
 ('cMEtAiW60I5wE_vLfTxoJQ', 1255)]

In [12]:
jsonRDD.map(json.loads).map(lambda x: (x['business_id'], 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda x: x[1], ascending=False).take(10)

[('4JNXUYY8wbaaDmk3BPzlWw', 8570),
 ('RESDUcs7fIiihp38-d6_6g', 8568),
 ('K7lWdNUhCbcnEvI0NhGewg', 6887),
 ('f4x1YBxkLrZg652xt2KR5g', 5847),
 ('cYwJA2A6I12KNkm2rtXd5g', 5575),
 ('DkYS3arLOhA8si5uUEmHOw', 5206),
 ('2weQS-RnoOBhb1KsHKyoSQ', 4534),
 ('5LNZ67Yw9RD6nf4_UhXOjw', 4522),
 ('iCQpiavjjPzJ5_3gPD5Ebg', 4351),
 ('SMPbvZLSMMb7KU76YNYMGg', 4350)]